# **💫🧙‍♂️ StyleGAN3 + CLIP** 

---


**Text-to-Image Portrait-Style Image Generation:** Faces, Metart, Animals, and Cosplay


( +) **StyleGAN3 + CLIP mishin_learning colab** version of based on @*nshepperd1* colab | inteface by [@nn_for_science](
https://t.me/nn_for_science) | and 
[@mishin_learning](https://t.me/mishin_learning) mod for *cut_size*, *cutn* and mainly *differentiable augmentations*.


( ++ ) **Addons by** [*UnendingBubbles*](https://github.com/unendingbubbles) : batch, history-view, grid-of-images, output sharpening, cutout_methods, color-to-alpha png, icon export, QOL everywhere, `Runtime > Run All` compliant 😁

*last updated: 211111*

Detailed instructions at the bottom.

mishin resources: [mishin original colab link](https://colab.research.google.com/drive/1LWWlW40-ZHnyhyJgU6xKaSsl8bwCT1wx?usp=sharing#scrollTo=FMwDxUXjoJJW)
 ~ [mishin blog](https://www.telemetr.me/content/mishin_learning) ~ [Телеграм-канал Мишин Лернинг ](https://t.me/mishin_learning) ~ 
[Transformer | Прогрессивное ml комьюнити  ](https://transformer.community/) ~ [Transformer | YouTube Канал с лекциями по Machine Learning](https://www.youtube.com/channel/UCJABbIsR2bd666zFMHzUFTQ)  

In [ ]:
# @title 📜 Licensed under the MIT License
# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  # print(gpu_info) #for more gpu info
  !nvidia-smi -L

In [ ]:
# @title 📘 Install Libraries
# @markdown This cell could take a while since it downloads several libraries
 
#limit maxHeight of output
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 176})'''))


!pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchtext>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/NVlabs/stylegan3
!git clone https://github.com/openai/CLIP
!pip install -e ./CLIP
!pip install einops ninja
!pip install lpips
!pip install kornia

import sys
sys.path.append('./CLIP')
sys.path.append('./stylegan3')

import io
import os, time
#more debug info
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import pickle
import shutil
import kornia.augmentation as K
import numpy as np
import math
from PIL import Image
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from IPython.display import display
from einops import rearrange

from google.colab import files
import feather
import pandas as pd
from google.colab import data_table
%load_ext google.colab.data_table 
 #restore native spreadsheet
# %unload_ext google.colab.data_table 

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

model notes:

| **ffhq** | **metfaces**  | **afhqv2** | **cosplay_faces** | | t  | r |
|:--------------:|:-----------:|:------------:|:----------:|:-----------:|:------------:|:----------:|
| photo faces | portrait painings | animal faces | by [@l4rz](https://twitter.com/l4rz) | ‎  | translation | rotation |

In [ ]:
#@markdown 🧹 Delete all files in `finals` folder, before continuing? (leave unchecked)

# %%js
# somethin = confirm("GAN3+CLIP_formated_batch-c2a-grid-ico Asks:\n\nAre you sure you want to delete all 'final' images? 🔥");
 
clear_finals_now = False #@param {type:"boolean"}

if clear_finals_now == True: 
   %rm /content/finals/*


In [ ]:
#@markdown #**💫 StyleGAN3+CLIP Set Parameters**
import numpy as np
base_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"
#%cd /content/

##//////////////////////////////////////////////
# 💫 User Parameters
#@markdown ___
#@markdown **🎭 Choose a model:**
model_name = "stylegan3-r-ffhq-1024x1024.pkl" #@param ["stylegan3-r-afhqv2-512x512.pkl", "stylegan3-r-ffhq-1024x1024.pkl", "stylegan3-r-ffhqu-1024x1024.pkl","stylegan3-r-ffhqu-256x256.pkl","stylegan3-r-metfaces-1024x1024.pkl","stylegan3-r-metfacesu-1024x1024.pkl","stylegan3-t-afhqv2-512x512.pkl","stylegan3-t-ffhq-1024x1024.pkl","stylegan3-t-ffhqu-1024x1024.pkl","stylegan3-t-ffhqu-256x256.pkl","stylegan3-t-metfaces-1024x1024.pkl","stylegan3-t-metfacesu-1024x1024.pkl", "stylegan2-cosplay-faces-512x512-px"]
network_url = base_url + model_name
#@markdown Enable `model_mixing` if you want to mix between two models.(matching resolutions)

model_mixing = True #@param {type:"boolean"}
model2_name = "stylegan3-r-metfaces-1024x1024.pkl" #@param ["stylegan3-r-ffhq-1024x1024.pkl", "stylegan3-r-ffhqu-1024x1024.pkl","stylegan3-r-metfaces-1024x1024.pkl","stylegan3-r-metfacesu-1024x1024.pkl","stylegan3-t-ffhq-1024x1024.pkl","stylegan3-t-ffhqu-1024x1024.pkl","stylegan3-t-metfaces-1024x1024.pkl","stylegan3-t-metfacesu-1024x1024.pkl"] 
#@markdown Slide `proportion` to the right, for more `model_name` influence.
proportion  = 0.985 #@param {type:"slider", min:0, max:1, step:0.005}
#@markdown ___
#@markdown **💫 Enter your text prompt here:** 

# text = 'A sexy blonde marilyn monroe pop art in style of andy warhol professional art' #mishin
text = 'Carl Sagan in space, and happy. Carl Sagan in the cosmic ocean.' #@param {type:"string"}


steps =  45#@param {type:"integer"}
n_batches = 12#@param {type:"integer"}
# @markdown (fix = less subject movement)
fix_coordinates = "False" #@param ["True", "False"]
# @markdown (-1 for random seed)
seed =  -1#@param {type:"integer"}
sample_filetype = "png" #@param ["jpg", "png"]


show_every_n_steps =  22#@param
stats_for_nerds = True #@param {type:"boolean"}

#@markdown ___
#@markdown **Advanced Settings:**

#@markdown Changing the `cutout_method` greatly influences how the image evolves over time.
cutout_method = "nsheppard" #@param ["mishin", "nsheppard"]


#@markdown 32-128 works well for `u_cutn`. mishin default of 80. Cut number affects on 'Creativity': less cutout will lead to more random/creative results, sometimes barely readable, while higher values create stable/complex results. 💀 If CUDA crash, change to 40 or less. 
u_cutn = 80 #@param {type:"integer"} 

#@markdown Values of `u_cut_pow` below 1 prioritize structure over detail, and vice versa for above 1. mishin default 0.1, torch default 1.0,  UnenBubb default 2.5 (highly dependent on you promt)
u_cut_pow = 1.0 #@param {type:"number"}  default 1.0. 0.1-3.0 work okay

#@markdown Step size is learning rate: mishin default of 0.03. torch default of 0.001
u_step_size = 0.03 #@param {type:"number"} default 0.03

#@markdown Something to do with setup random connections/loss. Default 8. 💀 If CUDA crash, use 4. 
u_t_ran = "8" #@param [4, 8]

#@markdown Betas: coefficients used for computing running averages of gradient and its square. torch default: 0.9, 0.999. mishin default: 0.0, 0.9. mishin alt: 0.0, 0.999 or 0.8, 0.9. 

u_beta_a = 0.0 #@param {type:"number"}

u_beta_b = 0.9 #@param {type:"number"}

#@markdown ___

save_into_history = True #@param {type:"boolean"}
clear_history = False #@param {type:"boolean"}




In [ ]:
#@markdown #**🏃‍♀️ Run StyleGAN3+CLIP**
#@markdown Images placed in `samples` and `finals` folder to your left.


# You can doubleclick the grey area to the right to hide code.
##//////////////////////////////////////////////
# Parameter Setup

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

def fetch_model(url_or_path):
    basename = os.path.basename(url_or_path)
    if os.path.exists(basename):
        return basename
    else:
        !wget -c '{url_or_path}'
        return basename

if model_name == "stylegan2-cosplay-faces-512x512-px":
    network_url = 'https://l4rz.net/cosplayface-snapshot-004000-18160-FID367.pkl'
    
with open(fetch_model(network_url), 'rb') as fp:
  G = pickle.load(fp)['G_ema'].to(device)

u_t_ran = int(u_t_ran)

seed_orig = seed
text_safe = text
text_safe = text_safe.replace(" ", "_")
# text_safe = text_safe.replace(" ", "%")


if fix_coordinates == "True":
    if model_name != "stylegan2-cosplay-faces-512x512-px":
        shift = G.synthesis.input.affine(G.mapping.w_avg.unsqueeze(0))
        G.synthesis.input.affine.bias.data.add_(shift.squeeze(0))
        G.synthesis.input.affine.weight.data.zero_()

# target = embed_url("https://4.bp.blogspot.com/-uw859dFGsLc/Va5gt-bU9bI/AAAAAAAA4gM/dcaWzX0ZxdI/s1600/Lubjana+dragon+1.jpg")
# target = embed_url("https://irc.zlkj.in/uploads/e399d2fee2c6edd9/20210827165231_0_nexus%20of%20abandoned%20places.%20trending%20on%20ArtStation.png")



# escaping random noise is comfortable for everyone
#limit maxHeight of output
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 495})'''))

##//////////////////////////////////////////////
# CLIP for GAN3 Setup

def norm1(prompt):
    "Normalize to the unit sphere."
    return prompt / prompt.square().sum(dim=-1,keepdim=True).sqrt()

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)

if cutout_method == 'nsheppard':
  class MakeCutouts(torch.nn.Module):
      def __init__(self, cut_size, cutn, cut_pow=1.):
          print(f'cutn:{cutn}, cut_pow:{cut_pow}')
          super().__init__()
          self.cut_size = cut_size
          self.cutn = cutn
          self.cut_pow = cut_pow

      def forward(self, input):
          sideY, sideX = input.shape[2:4]
          max_size = min(sideX, sideY)
          min_size = min(sideX, sideY, self.cut_size)
          cutouts = []
          for _ in range(self.cutn):
              size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
              offsetx = torch.randint(0, sideX - size + 1, ())
              offsety = torch.randint(0, sideY - size + 1, ())
              cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
              cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
          return torch.cat(cutouts)

if cutout_method == 'mishin':
  class MakeCutouts(torch.nn.Module):
      def __init__(self, cut_size, cutn, cut_pow=1.):
          print(f'cutn:{cutn}, cut_pow:{cut_pow}')
          super().__init__()
          self.cut_size = cut_size
          self.cutn = cutn
          self.cut_pow = cut_pow
          self.augs = torch.nn.Sequential(
              K.RandomHorizontalFlip(p=0.5),
              K.RandomSharpness(0.3,p=0.4),
              K.RandomAffine(degrees=7, translate=0.05, p=0.7, padding_mode='border'),
              K.RandomPerspective(0.2, p=0.5),
              K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))

      def forward(self, input):
          sideY, sideX = input.shape[2:4]
          max_size = min(sideX, sideY)
          min_size = min(sideX, sideY, self.cut_size)
          cutouts = []
          for _ in range(self.cutn):
              size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
              offsetx = torch.randint(0, sideX - size + 1, ())
              offsety = torch.randint(0, sideY - size + 1, ())
              cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
              # cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
              cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
          batch = self.augs(torch.cat(cutouts, dim=0))
          return batch


make_cutouts = MakeCutouts(224, u_cutn, u_cut_pow)

def embed_image(image):
  n = image.shape[0]
  cutouts = make_cutouts(image)
  embeds = clip_model.embed_cutout(cutouts)
  embeds = rearrange(embeds, '(cc n) c -> cc n c', n=n)
  return embeds

def embed_url(url):
  image = Image.open(fetch(url)).convert('RGB')
  return embed_image(TF.to_tensor(image).to(device).unsqueeze(0)).mean(0).squeeze(0)

class CLIP(object):
  def __init__(self):
    clip_model = "ViT-B/32"
    self.model, _ = clip.load(clip_model)
    self.model = self.model.requires_grad_(False)
    self.normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                          std=[0.26862954, 0.26130258, 0.27577711])

  @torch.no_grad()
  def embed_text(self, prompt):
      "Normalized clip text embedding."
      return norm1(self.model.encode_text(clip.tokenize(prompt).to(device)).float())

  def embed_cutout(self, image):
      "Normalized clip image embedding."
      return norm1(self.model.encode_image(self.normalize(image)))
  
clip_model = CLIP()
target = clip_model.embed_text(text) 

if model_mixing == True:
  with open(fetch_model(base_url + model_name), 'rb') as fp:
      G1 = pickle.load(fp)['G_ema'].to(device)

  with open(fetch_model(base_url + model2_name), 'rb') as fp:
      G2 = pickle.load(fp)['G_ema'].to(device)

  G = G1
  for p_out, p_in1, p_in2 in zip(G.parameters(), G1.parameters(), G2.parameters()):
      p_out.data = torch.nn.Parameter(p_in1*proportion+p_in2*(1-proportion));
  print(f'model mixing: {model_name} x {model2_name} [{proportion}:{1-proportion}]')
else:
  print(f'model: {model_name}')

zs = torch.randn([10000, G.mapping.z_dim], device=device)
w_stds = G.mapping(zs, None).std(0)


##//////////////////////////////////////////////
# Generate image

from IPython.display import display

def batching(timestring):

  if seed_orig == -1:
    seed = np.random.randint(0, 2**32 - 1)
    
  tf = Compose([
    Resize(224),
    lambda x: torch.clamp((x+1)/2,min=0,max=1),
    ])

  torch.manual_seed(seed)
  # timestring = 'generation' #   timestring = time.strftime('%Y%m%d%H%M%S')
  rand_z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.mapping.z_dim)).to(device)
  q = (G.mapping(rand_z, None, truncation_psi=0.2)) / w_stds
  q.requires_grad_()
    
  with torch.no_grad():
      qs = []
      losses = []
      nn_loop = tqdm(range(0, 4))
      for _ in nn_loop:
          # q = (G.mapping(torch.randn([8, G.mapping.z_dim], device=device), None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds #if memory issue, change 8 to 4
          q = (G.mapping(torch.randn([u_t_ran, G.mapping.z_dim], device=device), None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds #if memory issue, change 8 to 4
          images = G.synthesis(q * w_stds + G.mapping.w_avg)
          embeds = embed_image(images.add(1).div(2))
          loss = spherical_dist_loss(embeds, target).mean(0)
          i = torch.argmin(loss)
          qs.append(q[i])
          losses.append(loss[i])
      qs = torch.stack(qs)
      losses = torch.stack(losses)
      i = torch.argmin(losses)
      q = qs[i].unsqueeze(0).requires_grad_()

  # Sampling loop
  q_ema = q
  #https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
  opt = torch.optim.AdamW([q], u_step_size, betas=(u_beta_a, u_beta_b))
  # opt = torch.optim.AdamW([q], lr=0.03, betas=(0.0, 0.9)) # 0.0, 0.999 or 0.8, 0.9, lr 0.02~0.04 def lr=0.03


  loop = tqdm(range(0, steps))
  for i in loop:
      nifty_name = f'{timestring}-s{str(seed)}-i{str(steps)}'
      opt.zero_grad()
      w = q * w_stds
      image = G.synthesis(w + G.mapping.w_avg, noise_mode='const')
      embed = embed_image(image.add(1).div(2))
      loss = spherical_dist_loss(embed, target).mean()
      loss.backward()
      opt.step()
      loop.set_postfix(loss=loss.item(), q_magnitude=q.std().item())

      q_ema = q_ema * 0.9 + q * 0.1  # was 0.9, 0.1
      image = G.synthesis(q_ema * w_stds + G.mapping.w_avg, noise_mode='const')

      if i % show_every_n_steps == 0:
          if stats_for_nerds == True:
            # kindloss = loss[6:]
            print(f"\n Image {i}/{steps} | Batch {k+1}/{n_batches} | Current loss: {loss} | {nifty_name}") 
          else:
            print('\n')
          display(TF.to_pil_image(tf(image)[0]))
      pil_image = TF.to_pil_image(image[0].add(1).div(2).clamp(0,1))
      os.makedirs(f'samples/{timestring}', exist_ok=True)
      pil_image.save(f'samples/{timestring}/{i:04}.{sample_filetype}')

      # insert save last numbered iteration
      if i == steps - 1:
        os.makedirs(f'finals', exist_ok=True)
        pil_image.save(f'finals/{nifty_name}.{sample_filetype}')
        # info_pass = timestring + "-i" + str(steps)
  if stats_for_nerds == True:         
    print(f"\n Image {i+1}/{steps} | Batch {k+1}/{n_batches} | Current loss: {loss} | {nifty_name}")  #added batch and info_pass 
  else:
    print(f'{nifty_name}')
  display(TF.to_pil_image(tf(image)[0]))
     
##//////////////////////////////////////////////
# Batching Loop

timestring = time.strftime('%Y%m%d%H%M%S')

print(text)

try:
  for k in range(n_batches): 
    info_pass = f'{timestring}-i{str(steps)}'
    timestring = time.strftime('%Y%m%d%H%M%S')
    batching(timestring)
except KeyboardInterrupt:  #ability to stop batch without unresponsive error
  pass

# Save images as a tar archive
!tar cf samples/{timestring}.tar samples/{timestring}
#@markdown ___

##//////////////////////////////////////////////
#History View

data_l = ['timestring','model_name','mixing','model2_name','proportion','text','steps','n_batches','fix','seed','sample_filetype','method','cutn','cut_pow','step_size','u_t_ran','u_beta_a','u_beta_b'] #column list
data_ev_l = [timestring,model_name,model_mixing,model2_name,proportion,text,steps,n_batches,fix_coordinates,seed,sample_filetype,cutout_method,u_cutn,u_cut_pow,u_step_size,u_t_ran,u_beta_a,u_beta_b]

from google.colab import data_table
# %load_ext google.colab.data_table 
import pandas as pd

try:
    big_history
except NameError:
    big_history = [] #initialize

if clear_history == True:
  big_history = []

if save_into_history == True:
  big_history.append(data_ev_l)
  # print(big_history)
  history_df = pd.DataFrame(big_history, columns=data_l) 

#tip: click here and press Ctrl+M+P (in that order) to scroll to top of cell
#     (or doubleclick the grey area to the right to hide code)

In [ ]:
#@markdown #⏳ History View: 💫 prompt
data_table.DataTable(history_df[["text", "timestring"]].tail(3), num_rows_per_page=3)

In [ ]:
#@markdown #⏳ History View: 💫 useful settings
data_table.DataTable(history_df[['model_name','mixing','model2_name','proportion','steps','n_batches','fix','seed','method','cutn','cut_pow','step_size']].tail(3), num_rows_per_page=3)

In [ ]:
#@markdown #**🔳 Create a Grid of Images**
#@markdown Files placed in `gridded` folder to your left.

from IPython.display import Image
import PIL, os, glob
from PIL import Image, ImageEnhance
from PIL import ImageDraw
from math import ceil, floor, trunc
from google.colab import files

generate_grid = True #@param {type:"boolean"}
destination = "/content/grided"
destination_presharp = "/content/finals-sharp"

filetype_in = "png" #@param ["jpg", "png"]
filetype_out = "jpg" #@param ["jpg", "png"]
#@markdown Enable `pixel_perfect` to disable resizing of images. If you want to set a custom final output size (frame_width), this must be disabled. Even with `pixel_perfect` enabled, images that are different aspect ratio or larger size from the first, will be resized to fit.
pixel_perfect = True #@param {type:"boolean"}
frame_width = 2000 #@param {type:"number"}
max_imgs =  32#@param {type:"number"}
images_per_row =  4#@param {type:"number"}
padding =  18#@param {type:"number"}
boarder = 40 #@param {type:"number"}
bgRed = 30 #@param {type:"number"} 
bgGreen = 30 #@param {type:"number"} 
bgBlue =  30#@param {type:"number"} 


#@markdown ✋ Warning: Session may crash if trying to display+download files above 5MB. Comment out the last lines to prevent that, when exporting large PNGs. You can manually download them from the `gridded` folder on the left.
download_when_complete = True  #@param {type:"boolean"}

#@markdown ___
#@markdown **Sharpening:** might only be useful if images are very downscaled, generated from non-sharpend datasets, or viewed on an imperfect medium. Sharpening is always a destructive process, but so is looking at a computer monitor.

#@markdown (Values above 1.00 sharpen, below 1.00 to blur.)

#@markdown **🌟 Pre-sharpening** to be applied to original input images, durring processing, before images are put into a grid. `pre_sharpening` is more useful durring downscaling.

apply_pre_sharp = False #@param {type:"boolean"}
#@markdown  (Values of 1.10-1.50 when downscaling. Less if `pixel_perfect` enabled)
pre_sharpening = 1.15 #@param {type:"number"}  
#@markdown  Save all those pre-sharpened originals in `finals-sharp` folder?
save_pre_sharps = False #@param {type:"boolean"}

#@markdown **🌟 Post-sharpening** to be applied 1:1 on final `gridded` result.
apply_post_sharp = False #@param {type:"boolean"}
#@markdown (Values of 1.05-1.40 work for viewing on screen. More for paper printing.)
post_sharpening = 1.10 #@param {type:"number"}  

#@markdown If you can notice the sharpening without zooming to 1:1 pixels, it's probably too much sharpening. Windows tip: press Ctrl+1 in windows-photo-viewer to view at 100% scale, after opening.


images = glob.glob("/content/finals/*." + filetype_in)
images = images[:max_imgs]

os.makedirs(f'{destination}', exist_ok=True)
full_destination = f'{destination}/{info_pass}-{text_safe}-{str(len(images))}up.{filetype_out}'

def gridcreator(destination, frame_width):

  images = glob.glob("/content/finals/*." + filetype_in)
  images = images[:max_imgs]                #get the first n# images
  images.sort(key=os.path.getctime)         #sort files by date


  img_width, img_height = Image.open(images[0]).size
  sf = (frame_width-(images_per_row-1)*padding)/(images_per_row*img_width)       #scaling factor
  scaled_img_width = ceil(img_width*sf)                  
  scaled_img_height = ceil(img_height*sf) + padding
  number_of_rows = ceil(len(images)/images_per_row)

  if pixel_perfect == True:
    scaled_img_width = img_width
    scaled_img_height = img_height + padding
    frame_width = images_per_row * (img_width) + ((images_per_row-1) * padding)

  #frame_height = ceil((sf*img_height+padding)*number_of_rows) 
  frame_height = ceil(scaled_img_height*number_of_rows)

  new_im = Image.new('RGB', (frame_width+boarder*2, frame_height+boarder*2-padding), (bgRed, bgGreen, bgBlue)) 

  i,j=0,0
  for num, im in enumerate(images):
      if num%images_per_row==0:
          i=0
      im = Image.open(im)

      if apply_pre_sharp == 1:
        enhancer = ImageEnhance.Sharpness(im)
        # enhanced_im = enhancer.enhance(pre_sharpening)
        im = enhancer.enhance(pre_sharpening)
        if save_pre_sharps == 1:
          os.makedirs(f'{destination_presharp}', exist_ok=True)
          if filetype_out == "jpg":
            new_im.save(f'{destination_presharp}/{info_pass}-{text_safe}-{i}', "JPEG", quality=80, optimize=True, progressive=True)
          if filetype_out == "png":
            new_im.save(f'{destination_presharp}/{info_pass}-{text_safe}-{i}', "PNG", quality=80, optimize=True, progressive=True)
        #im = enhanced_im
      im.thumbnail((scaled_img_width,scaled_img_height), resample=3, reducing_gap=3)
  #    im.thumbnail((scaled_img_width,scaled_img_height))
      y_cord = (j//images_per_row)*scaled_img_height
      new_im.paste(im, (i+boarder,y_cord+boarder))
      #print(i, y_cord)
      i=(i+scaled_img_width)+padding
      j+=1

  # TODO enable text
  # Call draw Method to add 2D graphics in an image
  #I1 = ImageDraw.Draw(new_im)
  # Add Text to an image
  #I1.text((28, 36), "nice Car", fill=(255, 0, 0))

  if apply_post_sharp == False:
    if filetype_out == "jpg":
      new_im.save(full_destination, "JPEG", quality=85, optimize=True, progressive=True)
    if filetype_out == "png":
      new_im.save(full_destination, "PNG", quality=85, optimize=True, progressive=True)
    if download_when_complete == True:
      print('download requet sent...')
      files.download(full_destination)
  else:
    enhancer = ImageEnhance.Sharpness(new_im)
    enhanced_im = enhancer.enhance(post_sharpening)

    if filetype_out == "jpg":
      enhanced_im.save(full_destination, "JPEG", quality=85, optimize=True, progressive=True)
    if filetype_out == "png":
      enhanced_im.save(full_destination, "PNG", quality=85, optimize=True, progressive=True)
    if download_when_complete == True:
      print('download requet sent...')
      files.download(full_destination)

  print(f"{full_destination} - {os.path.getsize(full_destination)/1048576:.3f} MB")

try:
  if generate_grid == True:
    #print('generating grid...')
    gridcreator(destination, frame_width)  #it gets upset if you dont pass these variables for some reason
    #Image(full_destination)
  else:
    print('check "generate_grid" to run this feature.')
except KeyboardInterrupt:  #ability to stop without unresponsive error
  pass

print(text)
full_destination = f'{destination}/{info_pass}-{text_safe}-{str(len(images))}up.{filetype_out}'
print("\n")
# from IPython.display import Image      #Colab will softcrash when displaying very large images
# Image(full_destination)

In [ ]:
#@markdown #💾💫 export current settings history into csv
#################################

export_to_csv = True #@param {type:"boolean"}

#@markdown ✋ If your runtime is restarted, the history will be cleared, and the csv will be overwritten as empty.

#@markdown If no `...history_df.csv` is found, one will be created.

if export_to_csv == True:
  import glob
  if not glob.glob('*ory_df.csv'): #if no csv exists, create one
    # history_df = ""  #do not set this to None...
    timestring4 = time.strftime('%Y%m%d-%H%M')
    history_csv_at = f'{timestring4}_history_df.csv'
    print(f'creating new history log: {history_csv_at}')
    history_df.to_csv(f'/content/{history_csv_at}')

  print(f'overwriting to history log: {history_csv_at}')
  history_df.to_csv(f'/content/{history_csv_at}')

#to import an old history, run this line
  # history_df = pd.read_csv('path_to_csv_file')

In [ ]:
#@title **🎥 Generate a `.mp4` video**
#@markdown Make a video of one batch. Files placed in `videos` folder to your left.

from PIL import ImageTk, Image
from tkinter import Tk
#https://stackoverflow.com/questions/61827899/attributeerror-image-has-no-attribute-open

from IPython import display
from base64 import b64encode
from google.colab import files

#User Parameters
generate_video = False #@param {type:"boolean"}
#@markdown Leave empty for last run. Path to images (`000x.jpg` / `000x.png`):
folder_of_images = '' #@param {type:"string"}
filetype_in = "jpg" #@param ["jpg", "png"]
#@markdown Desired time of the video in seconds (min fps is 10, so your video might be shorter then desired):
length =  5#@param 
#@markdown Leave unchecked for full length video. Frames to start and stop.
custom_frames = False #@param {type:"boolean"}
init_frame = 1 #@param {type:"integer"}
last_frame = 4 #@param {type:"integer"}
download_video_when_complete = False  #@param {type:"boolean"}

#Generate Video
data_url = None #must declare, or else HTML parse throws error when generate_video disable

if folder_of_images != '':
  directory_in = folder_of_images
  timestring = folder_of_images[-14:]
else:
  directory_in = f'samples/{timestring}'

if custom_frames == True:
  total_frames = last_frame - init_frame
else:
  total_frames = steps

min_fps = 10
max_fps = 30

if generate_video == True:
  frames = []
  print('Generating video...')
  if custom_frames == True:
    for i in range(init_frame,last_frame): 
      filename = f"{directory_in}/{i:04}.{filetype_in}"
      frames.append(Image.open(filename))
  else:
    for i in sorted(os.listdir(directory_in)): 
        frames.append(Image.open(f'{directory_in}/{i}'))

  fps = np.clip(total_frames/length,min_fps,max_fps)

  os.makedirs(f'videos', exist_ok=True)
  savepath=f'videos/{timestring}.mp4'
  from subprocess import Popen, PIPE
  p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', savepath], stdin=PIPE)
  for im in tqdm(frames):
      im.save(p.stdin, 'PNG')
  p.stdin.close()
  p.wait()

  print(f"{savepath} - {os.path.getsize(full_destination)/1048576:.3f} MB")
  print(text)
  print("\n")

  if download_video_when_complete == True:
    files.download(full_destination)

  mp4 = open(savepath,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@markdown 🧹 Delete all files in `transparent` folder, before continuing? (leave unchecked)

# %%js
# somethin = confirm("GAN3+CLIP_formated_batch-c2a-grid-ico Asks:\n\nAre you sure you want to delete all 'final' images? 🔥");
 
clear_transparent_now = False #@param {type:"boolean"}

if clear_transparent_now == True: 
   %rm /content/transparent/*

In [ ]:
#@markdown #**🔬Color Range to Alpha**
import os
import fnmatch

#@markdown Everything close to chosen color becomes transparent. Processes everything in `path_to_originals` folder into new `.png` in `tranparent` folder to your left.  
#Todo: add more options. fix minor ghosting.

generate_transparents = False #@param {type:"boolean"}
path_to_originals = '/content/finals/' #@param {type:"string"}
c_range = 120 #@param {type:"integer"}

cRed = 0 #@param {type:"number"} 
cGreen = 0 #@param {type:"number"} 
cBlue = 0#@param {type:"number"} 

if generate_transparents == True: 

  for path,dirs,files in os.walk('/content/finals/'):
      for file in files:
          if fnmatch.fnmatch(file,'*.*'):
              fullname = os.path.join(path,file)
              #print(fullname)
              lessname = file[:-4]

              #https://www.geeksforgeeks.org/how-to-make-background-image-transparent-using-python/
              #happy if-style color to alpha3
              from PIL import Image
                
              img = Image.open(fullname)
              rgba = img.convert("RGBA")
              datas = rgba.getdata()
                
              newData = []
              for item in datas:
                  #if item[0] == 0 and item[1] == 0 and item[2] == 0:  # finding black colour by its RGB value #default
                      # storing a transparent value when we find a black colour #default
                      #newData.append((255, 255, 255, 0)) #default
                  # if item[0] <= c_range and item[1] <= c_range and item[2] <= c_range:  # finding black colour by its RGB value
                  #if abs(item[0]-cRed) <= c_range and abs(item[1]-cBlue) <= c_range and abs(item[2])-cBlue <= c_range:  # finding black colour by its RGB value
                  if abs(item[0]-cRed) <= c_range or abs(item[1]-cBlue) <= c_range or abs(item[2])-cBlue <= c_range:  # finding black colour by its RGB value

                      # storing a transparent value when we find a black colour
                      # avgy = int(((item[0] + item[1] + item[2])/3) * (255/c_range))
                      avgy = int(((abs(item[0]-cRed) + abs(item[1]-cBlue) + abs(item[2])-cBlue)/3) * (255/c_range))
                      newData.append((item[0], item[1], item[2], avgy))
                  else:
                      newData.append(item)  # other colours remain unchanged
                
              rgba.putdata(newData)
              os.makedirs(f'transparent', exist_ok=True)
              savepath = "/content/transparent/" + lessname + "transp.png"
              rgba.save(savepath, "PNG")
              #print(savepath)

In [ ]:
#@markdown 🧹 Delete all files in `icons` folder, before continuing? (leave unchecked)

# %%js
# somethin = confirm("GAN3+CLIP_formated_batch-c2a-grid-ico Asks:\n\nAre you sure you want to delete all 'final' images? 🔥");
 
clear_icons_now = False #@param {type:"boolean"}

if clear_icons_now == True: 
   %rm /content/icons/*

In [ ]:
#@markdown #**📐 Create .ico files**
#@markdown Generate icons for your desktop 😁
#@markdown These `.ico` files contain everything from 16x16 to 256x256. Files placed in `icons` and `icons_zipped` folders to your left.

import os
import fnmatch

generate_icons = False #@param {type:"boolean"}
zip_icons = True #@param {type:"boolean"}

if generate_icons == True:

  timestring = time.strftime('%Y%m%d%H%M%S')
  os.makedirs(f'icons', exist_ok=True)
  for path,dirs,files in os.walk('/content/transparent/'):
      for file in files:
          if fnmatch.fnmatch(file,'*.png'):
              fullname = os.path.join(path,file)
              #print(fullname)
              lessname = file[:-4]

              from PIL import Image
              img = Image.open(fullname)
              savepath = "/content/icons/" + lessname + "icon.ico"
              img.save(savepath)
              #print(savepath)
  if zip_icons == True:
    os.makedirs(f'icons_zipped', exist_ok=True)
    import shutil
    shutil.make_archive(f'/content/icons_zipped/icons_{timestring}', 'zip', '/content/icons/')

In [ ]:
#@markdown other history-view options
# data_table.DataTable(history_df[["text", "timestring"]], include_index=False, num_rows_per_page=15)
# history_df[["text", "timestring"]].tail(2)
# history_df[["text", "timestring"]]
# history_df.head()
# history_df.tail()
# data_table.DataTable(history_df, num_rows_per_page=15)
# data_table.DataTable(history_df[["text", "timestring"]], include_index=False, num_rows_per_page=15)

In [ ]:
#@markdown tools to change history-view structure
# argys = 'timestring,model_name,model_mixing,model2_name,proportion,text,steps,n_batches,fix_coordinates,seed,sample_filetype,cutout_method,u_cutn,u_cut_pow,u_step_size,u_t_ran,u_beta_a,u_beta_b' #an unquoted-variable string in quotes

# #tool to create quoted variable list from string of nonquoted variables. 
# argys_quoted = argys.replace(",", "','") #add '' around commas
# argys_quoted = f"'{argys_quoted}'" #add '' on outside
# #results can be manually placed into variables above. useful for custom title list and to help maintain spreadsheet formatting (avoiding dict for now)
# print(argys_quoted) #goes in data_l (you can change names for custom titles)
# print(argys)  #goes in data_ev_l

✋ Note: this notebook likes to crash often.

Create an editable copy for yourself with `File > Save a Copy in Drive`.

**To Run this Colab Notebook**, select `Runtime` in the top toolbar menu, then `Run All (Ctrl+F9)`.   
  
**To Run/Stop individual Code Blocks:** Click the `▶` or `⏹` button in the upper-left of each code block. It will appear when hovering. These buttons are only functional at the top of the code block, and if you scroll too far down (e.g. inside the results-output), they become non-functional. `spinning` button means the block is running and `checkerd-spinning` means the block is queued.
  
You can doublecheck settings with `Runtime > Change Runstime Type`:  
* `Runtime Shape` should be `High-RAM` if you want to use a `cutn = 128` for the most detailed output. 
* `☑ Omit code cell output when saving this notebook` should be checked so that your `.ipynb` files do not grow massive in size.

Sometimes, code may fail to run correctly. Colab will also softcrash if your network cannot keep up with the stream of (uncompressed) images. Try `▶` running the cell-block again usually fixes this. If Google Colab is still acting weird, try a refresh with `F5` or setting a larger `show_every_n_steps`. If that does not work, try `Runtime > Restart Runtime` or `Runtime > Factory Restart Runtime`. If it is still fails, try a selecting a different `model`, a different `cutout_method` or a shorter `text` prompt.

Occationally, this code will give a "client side CUDA error", in which case, a factory reset and Run All with identical settings (and a new random seed) seems to fix.
  
